<a href="https://colab.research.google.com/github/indukrishna572/literate-broccoli/blob/main/TNMajorEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing all the necessary dependencies
# utilities
import re
import nltk
import numpy as np
import pandas as pd
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
#tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import sequence
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Importing the dataset
data_columns=['target','ids','date','flag','user','text']
data_encoding = "ISO-8859-1"
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', engine = 'python',encoding= data_encoding, names=data_columns, error_bad_lines=False)
data.sample(5)

<ipython-input-19-5f6e70a3e349>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', engine = 'python',encoding= data_encoding, names=data_columns, error_bad_lines=False)


,target,ids,date,flag,user,text
82772,0,1753161242,Sun May 10 00:22:53 PDT 2009,NO_QUERY,jennarathke,Last night at the 7207
13626,0,1553330429,Sat Apr 18 13:58:49 PDT 2009,NO_QUERY,XFaintheartedX,"@iamjonathancook Oh. Well, why would they do ..."
86697,0,1754201008,Sun May 10 05:23:29 PDT 2009,NO_QUERY,alecrivera,"will be sad from Tuesday (May 12, 2009) until ..."
197273,0,1970967780,Sat May 30 05:49:04 PDT 2009,NO_QUERY,Elliethinks,@hayleycoll i was while i was showering but it...
161436,0,1957303737,Fri May 29 00:06:03 PDT 2009,NO_QUERY,AnniesBangkok,Today is going to be a normal day for I hope. ...


In [ ]:
#STEP 2
#EXPLORATORY DATA ANALYSIS

#2.1 five top records of data (can be altered accordingly)
#head() returns the first 5 rows if a number is not specified
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#2.2 columns/features in data
data. columns #returns the label of each column in the dataframe

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [ ]:
#2.3 length of the dataset
print('length of the data is', len(data))

length of the data is 226860


In [ ]:
#2.4 shape of dataset
data.shape
#used to get the current shape of an array
#611831 rows and 6 columns

(226860, 6)

In [ ]:
#2.5 data information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226860 entries, 0 to 226859
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   target  226860 non-null  int64 
 1   ids     226860 non-null  int64 
 2   date    226860 non-null  object
 3   flag    226859 non-null  object
 4   user    226859 non-null  object
 5   text    226859 non-null  object
dtypes: int64(2), object(4)
memory usage: 10.4+ MB


In [ ]:
#2.6 datatype of all columns
data.dtypes


target     int64
ids        int64
date      object
flag      object
user      object
text      object
dtype: object

In [ ]:
#2.7 checking for null values
np.sum(data.isnull().any(axis=1)) #no of null values in columns

1

In [ ]:
newdata = data.dropna()

In [ ]:
#2.8 rows and columns in the dataset
print('Count of columns in the data is:  ', len(newdata.columns))
print('Count of rows in the data is:  ', len(newdata))

Count of columns in the data is:   6
Count of rows in the data is:   226859


In [ ]:
#2.9 checking unique target values
newdata['target'].unique()

array([0, 4])

In [ ]:
newdata['target'].nunique() #no of unique values

2

In [ ]:
#3 DATA PREPARATION

newdata=newdata[['text','target']]


In [ ]:
newdata_pos = newdata[newdata['target']==1]
newdata_neg = newdata[newdata['target']==0]

In [ ]:
#taking 1/4th data
#for ease of running it in the machine

newdata_pos = newdata_pos.iloc[:int(20000)]
newdata_neg = newdata_neg.iloc[:int(20000)]


In [ ]:
#combining positive and negative tweets
newdata = pd.concat([newdata_pos, newdata_neg])

In [ ]:
newdata['text'] = newdata['text'].str.lower()

In [ ]:
newdata['text'].tail()

19995    not much time off this weekend, work trip to m...
19996                            one more day of holidays 
19997    feeling so down right now .. i hate you damn h...
19998    geez,i hv to read the whole book of personalit...
19999    i threw my sign at donnie and he bent over to ...
Name: text, dtype: object

19995    not much time off this weekend, work trip to m...
19996                            one more day of holidays 
19997    feeling so down right now .. i hate you damn h...
19998    geez,i hv to read the whole book of personalit...
19999    i threw my sign at donnie and he bent over to ...
Name: text, dtype: object

In [ ]:
import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stopwords_list = stopwords.words('english')


In [ ]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
#cleaning and removing stop words from tweets
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
newdata['text'] = newdata['text'].apply(lambda text: cleaning_stopwords(text))
newdata['text'].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    upset can't update facebook texting it... migh...
2    @kenichan dived many times ball. managed save ...
3                     whole body feels itchy like fire
4    @nationwideclass no, behaving all. i'm mad. he...
Name: text, dtype: object

In [ ]:
#cleaning and removing punctuations
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

newdata['text']= newdata['text'].apply(lambda x: cleaning_punctuations(x))
newdata['text'].tail()

19995    much time weekend work trip malmï¿½ frisat tod...
19996                                     one day holidays
19997                     feeling right  hate damn humprey
19998    geezi hv read whole book personality types emb...
19999     threw sign donnie bent get thingee made sad face
Name: text, dtype: object

In [ ]:
#cleaning and removing reccurring chars
def cleaning_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
newdata['text'] = newdata['text'].apply(lambda x: cleaning_repeating_char(x))
newdata['text'].tail()

19995    much time wekend work trip malmï¿½ frisat toda...
19996                                     one day holidays
19997                       feling right hate damn humprey
19998    gezi hv read whole bok personality types embar...
19999       threw sign donie bent get thinge made sad face
Name: text, dtype: object

In [ ]:
def cleaning_URLs(newdata):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',newdata)


In [ ]:
newdata['text'] = newdata['text'].apply(lambda x: cleaning_URLs(x))
newdata['text'].tail()

19995    much time wekend work trip malmï¿½ frisat toda...
19996                                     one day holidays
19997                       feling right hate damn humprey
19998    gezi hv read whole bok personality types embar...
19999       threw sign donie bent get thinge made sad face
Name: text, dtype: object

In [ ]:
def cleaning_numbers(newdata):
    return re.sub('[0-9]+', '', newdata)


In [ ]:
newdata['text'] = newdata['text'].apply(lambda x: cleaning_numbers(x))
newdata['text'].tail()

19995    much time wekend work trip malmï¿½ frisat toda...
19996                                     one day holidays
19997                       feling right hate damn humprey
19998    gezi hv read whole bok personality types embar...
19999       threw sign donie bent get thinge made sad face
Name: text, dtype: object

In [ ]:
#TOKENIZATION OF TWEET TEXT

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
newdata['text'] = newdata['text'].apply(tokenizer.tokenize)
newdata['text'].head()

In [ ]:
st = nltk.PorterStemmer()
def stemming_on_text(newdata):
    text = [st.stem(word) for word in newdata]
    return newdata

newdata['text']= newdata['text'].apply(lambda x: stemming_on_text(x))

newdata['text'].head()


0    [switchfot, htptwitpicomyzl, aw, thats, bumer,...
1    [upset, cant, update, facebok, texting, it, mi...
2    [kenichan, dived, many, times, bal, managed, s...
3               [whole, body, fels, itchy, like, fire]
4    [nationwideclas, no, behaving, al, im, mad, he...
Name: text, dtype: object

In [ ]:
  >>> import nltk
  >>> nltk.download('wordnet')


lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(newdata):
    text = [lm.lemmatize(word) for word in newdata]
    return newdata

newdata['text'] = newdata['text'].apply(lambda x: lemmatizer_on_text(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
newdata['text'].head()


0    [switchfot, htptwitpicomyzl, aw, thats, bumer,...
1    [upset, cant, update, facebok, texting, it, mi...
2    [kenichan, dived, many, times, bal, managed, s...
3               [whole, body, fels, itchy, like, fire]
4    [nationwideclas, no, behaving, al, im, mad, he...
Name: text, dtype: object

In [ ]:
#separating input and label

X=newdata.text
y=newdata.target

In [ ]:
#Preparing the input features for training

max_len = 500
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix.shape


(20000, 500)

In [ ]:
#Separating the 70% data for training data and 30% for testing data

X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, y, test_size=0.3, random_state=2)


In [ ]:
#TRAINING THE MODEL
#using Tensorflow based model

def tensorflow_based_model(): #Defined tensorflow_based_model function for training tenforflow based model
    inputs = Input(name='inputs',shape=[max_len])#step1
    layer = Embedding(2000,50,input_length=max_len)(inputs) #step2
    layer = LSTM(64)(layer) #step3
    layer = Dense(256,name='FC1')(layer) #step4
    layer = Activation('relu')(layer) # step5
    layer = Dropout(0.5)(layer) # step6
    layer = Dense(1,name='out_layer')(layer) #step4 again but this time its giving only one output as because we need to classify the tweet as positive or negative
    layer = Activation('sigmoid')(layer) #step5 but this time activation function is sigmoid for only one output.
    model = Model(inputs=inputs,outputs=layer) #here we are getting the final output value in the model for classification
    return model #function returning the value when we call it

In [ ]:
#MODEL BUILDING

#calling the model
#using 2 classes hence using "binary_crossentropy"
#if we use more than two classes then we use "categorical_crossentropy"
#Optimizer is a function that used to change the features of neural network such as learning rate (how the model learn with features)
#in order to reduce the losses.
#We are setting metrics=accuracy
#because we are going to caluclate the percentage of correct predictions over all predictions on the validation set

model = tensorflow_based_model() # here we are calling the function of created model
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])


In [ ]:
#feeding the training data
#by set the following parameters:
#Batch size =80
#so the model take 80 tweets in each iteration and train them.
#Batch size is a term used in machine learning and refers to the number of training examples utilized in one iteration.
#Epochs =6 so the model will train on the data 6 times.
#Epoch is a term used in machine learning and indicates the number of passes of the entire training dataset the machine learning algorithm has completed.
#We can choose batch_size, and epochs as we want
# if the model will not give the good results we can change it and then try again for the training of the model.

In [ ]:
history=model.fit(X_train,Y_train,batch_size=80,epochs=6, validation_split=0.1)# here we are starting the training of model by feeding the training data
print('Training finished !!')

Epoch 1/6
158/158 [==============================] - 87s 526ms/step - loss: 0.0257 - accuracy: 0.9951 - val_loss: 1.2491e-05 - val_accuracy: 1.0000
Epoch 2/6
158/158 [==============================] - 86s 544ms/step - loss: 1.7731e-05 - accuracy: 1.0000 - val_loss: 3.8561e-06 - val_accuracy: 1.0000
Epoch 3/6
158/158 [==============================] - 77s 485ms/step - loss: 8.6365e-06 - accuracy: 1.0000 - val_loss: 2.1086e-06 - val_accuracy: 1.0000
Epoch 4/6
158/158 [==============================] - 80s 505ms/step - loss: 5.7414e-06 - accuracy: 1.0000 - val_loss: 1.3938e-06 - val_accuracy: 1.0000
Epoch 5/6
158/158 [==============================] - 77s 487ms/step - loss: 4.0088e-06 - accuracy: 1.0000 - val_loss: 1.0334e-06 - val_accuracy: 1.0000
Epoch 6/6
158/158 [==============================] - 80s 505ms/step - loss: 3.4219e-06 - accuracy: 1.0000 - val_loss: 8.0103e-07 - val_accuracy: 1.0000
Training finished !!


In [ ]:
#Testing the Trained model on test data
#Getting predictions/classifying the sentiments (positve/negative) on the test data using trained model.

accr1 = model.evaluate(X_test,Y_test) #we are starting to test the model here


188/188 [==============================] - 16s 83ms/step - loss: 8.0106e-07 - accuracy: 1.0000


In [ ]:
#Accuracy is the number of correctly classify tweets from all the tweets of positive and negative.

print('Test set\n  Accuracy: {:0.2f}'.format(accr1[1])) #the accuracy of the model on test data is given below

Test set
  Accuracy: 1.00
